## Middleware

Middleware lets you insert code specific to your agent at key points in the ReAct loop

<img src="./assets/middleware.png" width="500">

### Dynamic Prompting

<img src="./assets/LC_DynamicPrompts.png" width="500">

### Setup

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    is_employee: bool
    db_ctx: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return the results."""

    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db_ctx
    
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"
    

In [6]:
SYSTEM_PROMPT_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step by step.
- When you need data, call the tool `execute_sql` with one SELECT query.
- Read-only only: no INSERT, UPDATE, DELETE, ALTER, DROP, CREATE, REPLACE, TRUNCATE statements.
- Limit to 5 rows unless the user specifies otherwise.
{table_limits}
- If the tool returns an error, fix the query and try again.
- Prefer explicit column lists instead of SELECT *.
"""

### Build a dynamic prompt

Utilize runtime context and middleware to generate a dynamic prompt

In [7]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt

@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    if not request.runtime.context.is_employee:
        table_limits = "- Limit access to these tables: Album. Artist, Genre, Playlist, PlaylistTrack, Track."
    else:
        table_limits = ""
        
    return SYSTEM_PROMPT_TEMPLATE.format(table_limits=table_limits)

Include middleware in create_agent

In [11]:
from langchain.agents import create_agent

agent = create_agent(
    model="ollama:gpt-oss:20b-cloud",
    tools=[execute_sql],
    middleware=[dynamic_system_prompt],
    context_schema=RuntimeContext,
)

In [12]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": {"role": "user", "content": question}},
    context=RuntimeContext(is_employee=False, db_ctx=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================

I’m sorry, but with the tables you’ve provided (Album, Artist, Genre, Playlist, PlaylistTrack, and Track) there’s no data about purchases, customers, or sales amounts. Consequently I can’t determine who made a purchase or how much it cost.

If you have another table that records transactions (e.g., a `Purchase`, `Order`, or `Invoice` table) and can grant me read access to it, I would be happy to look up the most expensive purchase made by Frank Harris.


Let's try for a employee

In [13]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": {"role": "user", "content": question}},
    context=RuntimeContext(is_employee=True, db_ctx=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (839eab76-bcac-4ff4-becc-75c2ea8c210c)
 Call ID: 839eab76-bcac-4ff4-becc-75c2ea8c210c
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (9d7e1b73-5be5-47d5-bd4d-2f590a451ff6)
 Call ID: 9d7e1b73-5be5-47d5-bd4d-2f590a451ff6
  Args:
    query: SELECT CustomerId, FirstName, LastName FROM Customer WHERE FirstName='Frank' AND LastName='Harris';
==============================